In [2]:
from astropy.io import ascii
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
import math

# read in file downloaded from SDSS
sdss_file = ascii.read('N_rich_QSOs_w_offsets.csv',format='csv')
 
# I had stupid naming convention in output file
# ra, dec: of bright offset star
# ra_deg, dec_deg: of QSO
offset_coords = SkyCoord(ra=sdss_file["ra"],dec=sdss_file["dec"],unit=(u.deg,u.deg))
qso_coords = SkyCoord(ra=sdss_file["ra_deg"],dec=sdss_file["dec_deg"],unit=(u.deg,u.deg))

In [107]:
# only keep 1 offset star, the one that's closest
[unique_srcs,unique_inds] = np.unique(sdss_file["name"],return_index=True)
offset_rmag_keep = np.empty(len(unique_srcs))

# set up arrays to hold sexagesimal RA and Dec of offset star
offset_ra_out  = []
offset_dec_out = []

offset_ra_out_deg  = np.empty(len(unique_srcs))
offset_dec_out_deg = np.empty(len(unique_srcs))

# set up arrays to hold offsets in RA and Dec
del_ra  = np.empty(len(unique_srcs))
del_dec = np.empty(len(unique_srcs))

# cycle through each source
for i in range(0,len(unique_srcs)):
    
    # find the offset stars associated with this star
    match = np.where(sdss_file["name"] == unique_srcs[i])
    
    # find the closest offset star
    dis  = qso_coords[unique_inds][i].separation(offset_coords[match]).arcsecond
    keep = np.where(dis == min(dis))
    
    # calculate offset from offset star TO target
    dra,ddec = offset_coords[match][keep].spherical_offsets_to(qso_coords[unique_inds][i])
    
    del_ra[i]  = np.round(dra.to(u.arcsec)/u.arcsec,1)
    del_dec[i] = np.round(ddec.to(u.arcsec)/u.arcsec,1)
    
    # for output file, save RA and Dec of preferred offset star in sexagesimal format    
    offset_ra_out.append(offset_coords.ra[match][keep].to_string(u.hourangle,sep=':',pad=True,precision=2))
    offset_dec_out.append(offset_coords.dec[match][keep].to_string(u.degree,sep=':',pad=True,precision=1))
    
    # to save file, use degrees, convert in topcat since I can't get sexagesimal format to save
    # and I need to move on with my life
    offset_ra_out_deg[i]  = offset_coords.ra[match][keep].deg
    offset_dec_out_deg[i] = offset_coords.dec[match][keep].deg
    
    offset_rmag_keep[i] = sdss_file["r"][match][keep]
    
offset_ra_out  = np.array(offset_ra_out)  
offset_dec_out = np.array(offset_dec_out)

In [111]:
# write output file 
#ascii.write([sdss_file["name"][unique_inds],offset_ra_out,offset_dec_out,offset_rmag_keep,\
#             qso_coords.ra[unique_inds].to_string(u.hourangle,sep=':',pad=True,precision=2),\
#             qso_coords.dec[unique_inds].to_string(u.deg,sep=':',pad=True,precision=1),\
#             sdss_file["imag"][unique_inds],\
#             sdss_file["comment"][unique_inds]],\
#             'N_rich_QSOs_offsets_calculated.txt',\
#            names=["Name","Offset_RA","Offset_Dec","r_mag",\
#                  "Targ_RA","Targ_Dec","i_mag","comment"],\
#            overwrite=True)

ascii.write([sdss_file["name"][unique_inds],offset_ra_out_deg,offset_dec_out_deg,np.round(offset_rmag_keep,2),\
             qso_coords.ra[unique_inds].deg,qso_coords.dec[unique_inds].deg,\
             sdss_file["imag"][unique_inds],del_ra,del_dec,\
             sdss_file["comment"][unique_inds]],\
             'N_rich_QSOs_offsets_calculated.txt',\
            names=["Name","Offset_RA","Offset_Dec","r_mag",\
                  "Targ_RA","Targ_Dec","i_mag","del_RA","Del_Dec","comment"],\
            overwrite=True,format="commented_header")


